In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model ="Gemma2-9b-It", groq_api_key= groq_api_key)

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage("Hello, I am Jivan and I am a Computer Engineer")])

AIMessage(content="It's nice to meet you, Jivan!  \n\nWhat kind of computer engineering are you interested in?  Hardware, software, networking, artificial intelligence?  I'm always eager to learn more about what people are working on in this field. \n\nTell me about yourself! What are you working on these days?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 21, 'total_tokens': 91, 'completion_time': 0.127272727, 'prompt_time': 0.000163629, 'queue_time': 0.012955331, 'total_time': 0.127436356}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8447398f-f953-4897-a804-6578f14523e8-0', usage_metadata={'input_tokens': 21, 'output_tokens': 70, 'total_tokens': 91})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hello, I am Jivan and I am a Computer Engineer"),
        AIMessage(content="That's great! It's nice to meet another computer engineer. What area of computer engineering are you most interested in?\n\nI'm still under development, but I'm learning about many different things related to computer engineering.  Do you have any questions for me or any projects you're working on that you'd like to discuss?\n"),
        HumanMessage(content ="Hey what is my name and what do I do ?")
    ]
)

AIMessage(content="You said your name is Jivan and you are a Computer Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 115, 'total_tokens': 154, 'completion_time': 0.070909091, 'prompt_time': 0.003554055, 'queue_time': 0.009726485, 'total_time': 0.074463146}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f5ad2856-a175-47f3-a74f-632844a7275a-0', usage_metadata={'input_tokens': 115, 'output_tokens': 39, 'total_tokens': 154})

In [7]:
from langchain_community.chat_message_histories import  ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import  RunnableWithMessageHistory

store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)


In [8]:
config = {"configurable":{"session_id":"chat1"}}

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, I am Jivan and I am a Computer Engineer")],
    config= config
)

In [10]:
response.content

"It's nice to meet you, Jivan! I'm Gemma, an AI assistant.  \n\nWhat are you working on these days? \n\nI'm always interested to hear about what people in Computer Engineering are up to.\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content="Hey what is my name?")],
     config=config

)

AIMessage(content='Your name is Jivan. 😊  I remember that from our first conversation! \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 87, 'total_tokens': 118, 'completion_time': 0.056363636, 'prompt_time': 0.003278547, 'queue_time': 0.012547803, 'total_time': 0.059642183}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cd6ad60-18f1-49a2-908a-bccd74f248ab-0', usage_metadata={'input_tokens': 87, 'output_tokens': 31, 'total_tokens': 118})

In [12]:
## changing the config --> session id
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hey what is my name?")],
     config=config1

)

response.content

"As an AI, I have no memory of past conversations and don't know your name. If you'd like to tell me, I'm happy to know! 😊  \n\n"

### With Prompt Templates

In [13]:
from langchain_core.prompts import  ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hello, I am Jivan")]})

AIMessage(content="Hello Jivan, it's nice to meet you!  \n\nHow can I help you today? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 31, 'total_tokens': 56, 'completion_time': 0.045454545, 'prompt_time': 0.000364869, 'queue_time': 0.01300692, 'total_time': 0.045819414}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7532b77c-62f0-47a7-b4a1-0145dd06c411-0', usage_metadata={'input_tokens': 31, 'output_tokens': 25, 'total_tokens': 56})

In [15]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [16]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hello, I am Jivan")],
    config= config,

)

response.content

"Hello Jivan!  It's nice to meet you.\n\nWhat can I do for you today? 😊  \n\n"

In [17]:
## adding complexity to the prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [18]:
response= chain.invoke({"messages":[HumanMessage(content="Hello, I am Jivan")],"language":"Nepali"})
response.content

'नमस्ते, जिवन! 👋 \n\nमलाई तपाईंले प्रश्न सोध्नुभयो भने म सबैभन्दा राम्रो प्रयास गर्नुहुन्छ।  \n \n\n के तपाईंलाई केही सोध्न चाहनुहुन्छ? 😊\n\n'

In [19]:
with_message_history = RunnableWithMessageHistory(chain, 
                                                  get_session_history,
                                                  input_messages_key="messages")

In [20]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="Hello, I am Jivan")],"language":"Nepali"},
    config= config
)
response.content


'नमस्ते, जिवन!  😊 \n\nमलाई तपाईंको प्रश्नहरुको उत्तर दिनुमा खुशी छु। के तपाईंलाई केही प्रश्न छ?  \n'

In [21]:
response = with_message_history.invoke(
    {"messages":[HumanMessage(content="What is my name?")],"language":"Nepali"},
    config= config
)
response.content


'आपको नाम जिवन हो।  \n'

### Managing  the conversation history


In [31]:
# trim_messages acts as trimmer for the messages

from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True, 
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="You are a helpful assistant"),

    HumanMessage(content="What’s the capital of France?"),
    AIMessage(content="The capital of France is Paris."),

    HumanMessage(content="Can you recommend a good science fiction book?"),
    AIMessage(content="I recommend reading 'Dune' by Frank Herbert."),

    HumanMessage(content="How do I boil an egg?"),
    AIMessage(content="To boil an egg, place it in a pot of water, bring it to a boil, then simmer for 9-12 minutes."),

    HumanMessage(content="Tell me a joke."),
    AIMessage(content="Why don't scientists trust atoms? Because they make up everything!"),

    HumanMessage(content="What’s the square root of 144?"),
    AIMessage(content="The square root of 144 is 12."),

    HumanMessage(content="Can you translate 'Hello' to Spanish?"),
    AIMessage(content="'Hello' in Spanish is 'Hola'."),

    HumanMessage(content="What’s the fastest land animal?"),
    AIMessage(content="The fastest land animal is the cheetah, capable of running up to 60-70 mph."),

    ]

trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="Can you translate 'Hello' to Spanish?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="'Hello' in Spanish is 'Hola'.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What’s the fastest land animal?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The fastest land animal is the cheetah, capable of running up to 60-70 mph.', additional_kwargs={}, response_metadata={})]

In [34]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What language did I ask to Translate?")],
    "language": "English"
    }
)
response.content

"You haven't asked me to translate anything yet.  \n\nIs there anything you'd like me to translate? 😊 \n"

In [35]:
response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What animal did you answer ?")],
    "language": "English"
    }
)
response.content

'I answered that the **cheetah** is the fastest land animal. 🐆 \n\n\nDo you have any other questions? 😊\n'

In [36]:
#Wrapping in message history

with_message_history = RunnableWithMessageHistory(chain, 
                                                  get_session_history,
                                                  input_messages_key="messages")

config = {"configurable":{"session_id":"chat5"}}

In [37]:
response = chain.invoke(
    {
    "messages": messages + [HumanMessage(content="What animal did you answer ?")],
    "language": "English"
    },
    config = config
)
response.content

'Cheetah! 🐆  \n\n\n\nI said the cheetah is the fastest land animal.  \n'